In [1]:
import os
import re
import json
import tempfile
import folium
import ipysheet
import numpy as np
import pandas as pd
import rasterio as rio
import rioxarray as rxr
import geopandas as gpd
import branca.colormap as cm
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import ipywidgets as widgets

from glob import glob
from pathlib import Path
from folium import plugins
from pyproj import Transformer
from ipyfilechooser import FileChooser
from ipysheet import from_dataframe
from rasterio.warp import calculate_default_transform, reproject, Resampling
from localtileserver import TileClient, get_leaflet_tile_layer, examples
from ipyleaflet import Map, Marker, basemaps, ScaleControl, LayersControl, AwesomeIcon, GeoData, basemap_to_tiles
from ipyleaflet import LegendControl, FullScreenControl, MarkerCluster

os.environ['LOCALTILESERVER_CLIENT_PREFIX'] = \
    f"{os.environ['JUPYTERHUB_SERVICE_PREFIX'].lstrip('/')}/proxy/{{port}}"

from localtileserver import TileClient, get_leaflet_tile_layer
from localtileserver import examples, helpers
from ipyleaflet import Map, SplitMapControl
import rasterio as rio

/tmp/ipykernel_932736/2179541366.py:11: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [12]:
chips = gpd.read_file('/explore/nobackup/people/mmacande/srlite/chm_model/20230926_chm_dtm/chips_df_v20230926.gpkg').to_crs('EPSG:4326')
chips['catid'] = chips['baseName'].str[-16:]
# chips
# chips_subset = chips
# chips_points = chips.copy()
# chips_points['geometry'] = chips_points['geometry'].centroid.to_crs('EPSG:4326')

# chips_subset_points.crs
# chips_subset_points
# type(chips_subset_points)
# chips_subset_points.to_crs('EPSG:4326').plot()
# chips

In [13]:
# catid = '10300100269C9900'
# test_fp = chmList1[chmList1['catid'] == catid]
# # test_fp = chmList2[chmList1['catid'] == '10300100269C9900']
# # test_fp
# # chips

# chips_img = chips[chips['catid'] == catid]
# chips_img
# chips_img_points = chips_img.copy()
# chips_img_points['geometry'] = chips_img['geometry'].centroid#.to_crs('EPSG:4326')

In [14]:
# chips['within_fp'] = chips.within(test_fp['geometry'].values[0])
# chips_other = chips[(chips['within_fp']) & (chips['catid'] != catid)]
# chips_other
# chips_other_points = chips_other.copy()
# chips_other_points['geometry'] = chips_other['geometry'].centroid#.to_crs('EPSG:4326')
# chips_other

In [15]:
# chips_img_geodata = GeoData(geo_dataframe = chips_img.to_crs('EPSG:4326'),
#                      style={'color': 'yellow', 'fillColor': '#3366cc', 'opacity':1, 'weight':2, 'dashArray':'2', 'fillOpacity':0.0},
#                      hover_style={'fillColor': 'red' , 'fillOpacity': 0.0},
#                      name = 'Chips from Image')

# chips_img_points_geodata = GeoData(geo_dataframe = chips_img_points.to_crs('EPSG:4326'),
#     style={'color': 'black', 'radius':8, 'fillColor': 'yellow', 'opacity':0.5, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
#     hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
#     point_style={'radius': 5, 'color': 'red', 'fillOpacity': 0.8, 'fillColor': 'blue', 'weight': 3},
#     name = 'Chip from Image Points')

# chips_other_geodata = GeoData(geo_dataframe = chips_other.to_crs('EPSG:4326'),
#                      style={'color': 'blue', 'fillColor': '#3366cc', 'opacity':1, 'weight':2, 'dashArray':'2', 'fillOpacity':0.0},
#                      hover_style={'fillColor': 'red' , 'fillOpacity': 0.0},
#                      name = 'Other Chips in Footprint')

# chips_other_points_geodata = GeoData(geo_dataframe = chips_other_points.to_crs('EPSG:4326'),
#     style={'color': 'black', 'radius':8, 'fillColor': 'blue', 'opacity':0.5, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
#     hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
#     point_style={'radius': 5, 'color': 'red', 'fillOpacity': 0.8, 'fillColor': 'blue', 'weight': 3},
#     name = 'Other Chips in Footprint Points')


In [16]:
# from ipyleaflet import Map, GeoJSON, GeoData
# import geopandas, pandas as pd, numpy as np

# m = Map(center=(62.5, -155), zoom=4)

# numpoints = 10
# center = (-155, 62.5)

# # df = pd.DataFrame(
# #     {'Conc': 1 * np.random.randn(numpoints) + 17,
# #      'Longitude': 0.0004 * np.random.randn(numpoints) + center[0],
# #      'Latitude': 0.0004 * np.random.randn(numpoints) + center[1]})

# # gdf = geopandas.GeoDataFrame(
# #     df, geometry=geopandas.points_from_xy(df.Longitude, df.Latitude))

# gdf = chips_subset_points.to_crs('EPSG:4326')

# geo_data = GeoData(geo_dataframe = gdf,
#     style={'color': 'black', 'radius':8, 'fillColor': '#3366cc', 'opacity':0.5, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
#     hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
#     point_style={'radius': 5, 'color': 'red', 'fillOpacity': 0.8, 'fillColor': 'blue', 'weight': 3},
#     name = 'Release')

# m.add_layer(geo_data)
# m


In [17]:
#https://girder.github.io/large_image/tilesource_options.html#style

# path = "/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/srlite/alaska/batch_1/4_band/split_3"
# filename = "WV02_20130813_M1BS_1030010026D4A600-sr-02m.tif"

ccdc_cir_style_list = [{'band': 4, 'palette': '#f00', 'min': 0, 'max': 6000}, {'band': 3, 'palette': '#0f0', 'min': 0, 'max': 2500}, {'band': 2, 'palette': '#00f', 'min': 0, 'max': 2500}]
ccdc_rgb_style_list = [{'band': 3, 'palette': '#f00', 'min': 0, 'max': 2500}, {'band': 2, 'palette': '#0f0', 'min': 0, 'max': 2500}, {'band': 1, 'palette': '#00f', 'min': 0, 'max': 2500}]

chm_palette = [
    '#63636300',
    '#fc8d59',
    '#fee08b','#fee08b',
    '#ffffbf','#ffffbf',
    '#d9ef8b','#d9ef8b',
    '#91cf60','#91cf60','#91cf60','#91cf60','#91cf60','#91cf60','#91cf60','#91cf60','#91cf60','#91cf60','#91cf60','#91cf60',
    '#1a9850','#1a9850','#1a9850','#1a9850','#1a9850','#1a9850','#1a9850','#1a9850','#1a9850','#1a9850','#1a9850','#1a9850','#1a9850','#1a9850','#1a9850','#1a9850','#1a9850','#1a9850','#1a9850','#1a9850',
    '#005a32'
]

# cloud_palette = ['#c2523c','#005a32']
chm10_style_list = [{'min': 0, 'max': 10.25, 'scheme': 'discrete', 'clamp': True, 'palette': chm_palette}]
cm_style_list = [{'palette': ['#00000000','#cccccc']}]
# chmpath = '/explore/nobackup/people/mmacande/srlite/chm_model/20230829_chm/products/WV02_20100804_M1BS_103001000617C000-sr-02m/WV02_20100804_M1BS_103001000617C000-sr-02m.cnn-chm-v1.tif'

lvis_RH098_path = '/explore/nobackup/people/mmacande/LVIS_annual/LVISF2_ABoVE2017_2019_RH098_mean_30m.tif'
lvis_RH098_client = TileClient(lvis_RH098_path)

In [ ]:
ifsar_dtm <- rast('/explore/nobackup/projects/dem/AK_IFSAR/alaska_ifsar_dtm_20221222.tif')
ifsar_dsm <- rast('/explore/nobackup/projects/dem/AK_IFSAR/alaska_ifsar_dsm_20221222.tif')


In [18]:
esri = basemap_to_tiles(basemaps.Esri.WorldImagery)
esri.base = True
carto = basemap_to_tiles(basemaps.CartoDB.Positron)
carto.base = True

In [19]:
def draw_chm_cir(row):
    chm_path = row['chm_path'] 
    srlite_path = row['srlite_path']
    toa_path = row['toa_path']
    # toa_path = '/explore/nobackup/people/mmacande/srlite/toa_30m/WV02_20130818_M1BS_10300100269C9900-toa.30m.cog.tif'
    ccdc_path = row['ccdc_path']
    cloudmask_path = row['cloudmask_path']
    chm_file = row['chm_file']
    catid = row['catid']
    fp = row['geometry']
    print(chm_path)
    print(srlite_path)
    print(toa_path)
    print(cloudmask_path)
    print(ccdc_path)
    print(fp)
    # test_fp = row['geometry']
    chips_img = chips[chips['catid'] == catid]
    chips_img
    chips_img_points = chips_img.copy()
    chips_img_points['geometry'] = chips_img['geometry'].to_crs('EPSG:3338').centroid.to_crs('EPSG:4326')

    chips['within_fp'] = chips.within(fp)#.values[0])
    chips_other = chips[(chips['within_fp']) & (chips['catid'] != catid)]
    chips_other
    chips_other_points = chips_other.copy()
    chips_other_points['geometry'] = chips_other['geometry'].to_crs('EPSG:3338').centroid.to_crs('EPSG:4326')
    chips_other

    print('Image chips: '+str(len(chips_img))+', Other chips: '+str(len(chips_other)))

    chips_img_geodata = GeoData(geo_dataframe = chips_img.to_crs('EPSG:4326'),
                         style={'color': 'black', 'fillColor': '#3366cc', 'opacity':1, 'weight':2, 'dashArray':'2', 'fillOpacity':0.0},
                         hover_style={'fillColor': 'red' , 'fillOpacity': 0.0},
                         name = 'Chips from Image')

    chips_img_points_geodata = GeoData(geo_dataframe = chips_img_points.to_crs('EPSG:4326'),
        style={'color': 'black', 'radius':8, 'fillColor': 'black', 'opacity':0.5, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
        hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
        point_style={'radius': 5, 'color': 'red', 'fillOpacity': 0.8, 'fillColor': 'blue', 'weight': 3},
        name = 'Chip from Image Points')

    chips_other_geodata = GeoData(geo_dataframe = chips_other.to_crs('EPSG:4326'),
                         style={'color': 'blue', 'fillColor': '#3366cc', 'opacity':1, 'weight':2, 'dashArray':'2', 'fillOpacity':0.0},
                         hover_style={'fillColor': 'red' , 'fillOpacity': 0.0},
                         name = 'Other Chips in Footprint')

    chips_other_points_geodata = GeoData(geo_dataframe = chips_other_points.to_crs('EPSG:4326'),
        style={'color': 'black', 'radius':8, 'fillColor': 'blue', 'opacity':0.5, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
        hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
        point_style={'radius': 5, 'color': 'red', 'fillOpacity': 0.8, 'fillColor': 'blue', 'weight': 3},
        name = 'Other Chips in Footprint Points')
    
    
    # nbands = rio.open(srlite_path).read().shape[0]
    nbands = rio.open(srlite_path).meta['count']
    if nbands == 8:
        srlite_cir_bands = [7,5,3]
        srlite_rgb_bands = [5,3,2]
        toa_cir_style_list = [{'band': 7, 'palette': '#f00', 'min': 0, 'max': 6000}, {'band': 5, 'palette': '#0f0', 'min': 0, 'max': 2500}, {'band': 3, 'palette': '#00f', 'min': 0, 'max': 2500}]
        toa_rgb_style_list = [{'band': 5, 'palette': '#f00', 'min': 0, 'max': 2500}, {'band': 3, 'palette': '#0f0', 'min': 0, 'max': 2500}, {'band': 2, 'palette': '#00f', 'min': 0, 'max': 2500}]
        srlite_cir_style_list = [{'band': 4, 'palette': '#f00', 'min': 0, 'max': 6000}, {'band': 3, 'palette': '#0f0', 'min': 0, 'max': 2500}, {'band': 2, 'palette': '#00f', 'min': 0, 'max': 2500}]
        srlite_rgb_style_list = [{'band': 3, 'palette': '#f00', 'min': 0, 'max': 2500}, {'band': 2, 'palette': '#0f0', 'min': 0, 'max': 2500}, {'band': 1, 'palette': '#00f', 'min': 0, 'max': 2500}]
    else:
        srlite_cir_bands = [4,3,2]
        srlite_rgb_bands = [3,2,1]
        toa_cir_style_list = [{'band': 4, 'palette': '#f00', 'min': 0, 'max': 6000}, {'band': 3, 'palette': '#0f0', 'min': 0, 'max': 2500}, {'band': 2, 'palette': '#00f', 'min': 0, 'max': 2500}]
        toa_rgb_style_list = [{'band': 3, 'palette': '#f00', 'min': 0, 'max': 2500}, {'band': 2, 'palette': '#0f0', 'min': 0, 'max': 2500}, {'band': 1, 'palette': '#00f', 'min': 0, 'max': 2500}]
        srlite_cir_style_list = [{'band': 4, 'palette': '#f00', 'min': 0, 'max': 6000}, {'band': 3, 'palette': '#0f0', 'min': 0, 'max': 2500}, {'band': 2, 'palette': '#00f', 'min': 0, 'max': 2500}]
        srlite_rgb_style_list = [{'band': 3, 'palette': '#f00', 'min': 0, 'max': 2500}, {'band': 2, 'palette': '#0f0', 'min': 0, 'max': 2500}, {'band': 1, 'palette': '#00f', 'min': 0, 'max': 2500}]
    print('nband:', nbands)
    
    
    chm_client = TileClient(chm_path)
    sr_client = TileClient(srlite_path)
    toa_client = TileClient(toa_path)
    if(ccdc_path): ccdc_client = TileClient(ccdc_path) 
    cm_client = TileClient(cloudmask_path)
    
    sr_cir = get_leaflet_tile_layer(
            sr_client, show=False, name = 'CIR SRLite',# bands=srlite_cir_bands,
            scheme='linear', dtype='uint16', style={'bands': srlite_cir_style_list})
    sr_rgb = get_leaflet_tile_layer(
            sr_client, show=False, name = 'RGB SRLite',# bands=srlite_rgb_bands,
            scheme='linear', dtype='uint16', style={'bands': srlite_rgb_style_list})
    toa_cir = get_leaflet_tile_layer(
            toa_client, show=False, name = 'CIR TOA',# bands=srlite_cir_bands,
            scheme='linear', dtype='uint16', style={'bands': toa_cir_style_list})
    toa_rgb = get_leaflet_tile_layer(
            toa_client, show=False, name = 'RGB TOA',# bands=srlite_rgb_bands,
            scheme='linear', dtype='uint16', style={'bands': toa_rgb_style_list})
    if(ccdc_path): ccdc_cir = get_leaflet_tile_layer(
            ccdc_client, show=False, name = 'CIR CCDC', bands=[4,3,2],
            scheme='linear', dtype='uint16', style={'bands': ccdc_cir_style_list}) 
    if(ccdc_path): ccdc_rgb = get_leaflet_tile_layer(
            ccdc_client, show=False, name = 'RGB CCDC', bands=[3,2,1],
            scheme='linear', dtype='uint16', style={'bands': ccdc_rgb_style_list}) 
    chm = get_leaflet_tile_layer(
        chm_client, show=True, name= 'CHM '+chm_file, clamp=True, 
        scheme='linear', dtype='uint16', style={'bands': chm10_style_list})
    lvis_RH098 = get_leaflet_tile_layer(
        lvis_RH098_client, show=False, name = 'lvis 2017 and 2019 RH098',
        scheme='linear', dtype='uint16', style={'bands': chm10_style_list})
    
    cm = get_leaflet_tile_layer(
        cm_client, show=False, name= 'cloudmask', style={'bands': cm_style_list})

    
    m = Map(center=chm_client.center(), zoom=chm_client.default_zoom, layers=[carto, esri], scroll_wheel_zoom=True)
    control = LayersControl(position='topright')
    # if(ccdc_path): m.add_layer(ccdc_rgb) 
    # m.add_layer(toa_rgb)
    # m.add_layer(sr_rgb)
    if(ccdc_path):  m.add_layer(ccdc_cir) 
    # m.add_layer(toa_cir)
    m.add_layer(sr_cir)
    m.add_layer(chm)
    m.add_layer(lvis_RH098)
    m.add_layer(cm)
    m.add_layer(chips_img_geodata)
    m.add_layer(chips_other_geodata)
    # m.add_layer(chips_img_points_geodata)
    # control = LayersControl(position='topright')
    # m.add_control(control)
    m.add_control(FullScreenControl())
    m.add_control(ScaleControl(imperial=False))
    m.add_control(LegendControl({
        '<0.25m': '#63636300',
        '<0.50m': '#fc8d59',
        '<1.0 m': '#fee08b',
        '<1.5 m': '#ffffbf',
        '<2.0 m': '#d9ef8b',
        '<5.0 m': '#91cf60',
        '<10 m': '#1a9850',
        '>10 m': '#005a32'},
        position='bottomleft'))
    m.add_control(control)

    m
    return m

        

In [20]:
from ipywidgets import Layout
defaultLayout=Layout(width='90%', height='800px')


In [27]:
chmList = gpd.read_file('/explore/nobackup/people/mmacande/srlite/chm_model/20230926_chm_dtm/chm_srlite_list_20230926.gpkg')
# chmList1 = pd.read_csv('/explore/nobackup/people/mmacande/srlite/chm_model/20230829_chm/products/chm_srlite_list_20230913.csv')
# chmList2 = pd.read_csv('/explore/nobackup/people/mmacande/srlite/chm_model/20230829_chm/products/chm_srlite_list_batch2_20230913.csv')
# chmList1 = gpd.read_file('/explore/nobackup/people/mmacande/srlite/chm_model/20230829_chm/products/chm_srlite_list_20230915.gpkg')
# chmList2 = gpd.read_file('/explore/nobackup/people/mmacande/srlite/chm_model/20230829_chm/products/chm_srlite_list_batch2_20230915.gpkg')
# print(len(chmList1))
# # chmList1 = chmList1[chmList1['ccdc_path'].notnull()]
# # chmList1 = chmList1[chmList1['ccdc_path'] != 'None']
# print(len(chmList2))

# chmList = pd.concat([chmList1, chmList2], ignore_index=True)
# chmList1[chmList1['catid'] == '1030010097573500']
print(len(chmList))
# chmList

53


In [28]:
# chmListNew = gpd.read_file('/explore/nobackup/people/mmacande/srlite/chm_model/20230916_chm/chm_srlite_list_20230918.gpkg')
# print(len(chmListNew))
# # chmListNew

In [35]:
# chmExample = chmList.sample(n=1)
chmExample = chmList.sample(n=1)
# chmExample = chmListNew[chmListNew['catid'] == '1030010006544A00']
# chmExample = chmList[chmList['catid'] == '104001002E6F3B00']
# chmExample = chmListNew[chmListNew['catid'] == '104001004F0ED700']

# chmExample = chmListNew.sample(n=1)
chmExample

# chips_intersect = chips.

map = chmExample.apply(draw_chm_cir, axis=1).values[0]
map.layout.width = '90%'
map.layout.height = '800px'

map

/explore/nobackup/people/mmacande/srlite/chm_model/20230926_chm_dtm/products_train_strips/WV02_20130807_M1BS_1030010025484800-sr-02m/WV02_20130807_M1BS_1030010025484800-sr-02m.cnn-chm-v1.tif
/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/srlite/alaska/batch_1/8_band_csv/split_6_1/WV02_20130807_M1BS_1030010025484800-sr-02m.tif
/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/toa/alaska/split_6_1/WV02_20130807_M1BS_1030010025484800-toa.tif
/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/cloudmask/alaska_batch_1/split_6/WV02_20130807_M1BS_1030010025484800-toa.cloudmask.tif
/explore/nobackup/people/mmacande/srlite/srlite_shared/ccdc_20230214/alaska/WV02_20130807_M1BS_1030010025484800-ccdc.tif
POLYGON ((-151.73217943302086 64.95885250356999, -151.6791644838079 66.02966267571213, -152.26922805082995 66.03358898725291, -152.29856614612706 64.9625897823302, -151.73217943302086 64.95885250356999))
Image chips: 467, Other chips: 150
nband: 8


Map(center=[65.49622074541145, -151.98886531496748], controls=(ZoomControl(options=['position', 'zoom_in_text'…

In [140]:
# chmList1
# print(chmList1['chm_path'].values[0])


In [132]:
# # client = TileClient(os.path.join(path, filename))
# print(chmList1['chm_path'].values[0])
# client = TileClient(chmList1['chm_path'].values[0])

# sigma = 2
# # style_list = []
# # for bid, pid in zip(data_bands, ['#f00', '#0f0', '#00f']):
# #     band_stats = client.rasterio.statistics(bid)
# #     print(bid, pid, band_stats)
# #     # newmin = band_stats.mean - (band_stats.std * sigma)
# #     # newmax = band_stats.mean + (band_stats.std * sigma)
# #     newmin = 0
# #     newmax = 2500
# #     style_list.append(
# #         {'band': bid, 'palette': pid, 'min': newmin, 'max': newmax})
# #     print(style_list)

# chm = get_leaflet_tile_layer(
#         client, show=True, name= 'CHM predicted', clamp=True, # band=data_bands,
#         scheme='linear', dtype='uint16', style={'bands': chm10_style_list}
# )

# # toa = get_leaflet_tile_layer(client)

# m = Map(
#     center=client.center(), zoom=client.default_zoom, basemap=basemaps.Esri.WorldImagery, scroll_wheel_zoom=True)
# m.add_layer(chm)
# control = LayersControl(position='topright')
# m.add_control(control)
# m.add_control(FullScreenControl())

# m

In [ ]:
# client = TileClient(fullpath)

# sigma = 2
# # style_list = []
# # for bid, pid in zip(data_bands, ['#f00', '#0f0', '#00f']):
# #     band_stats = client.rasterio.statistics(bid)
# #     print(bid, pid, band_stats)
# #     # newmin = band_stats.mean - (band_stats.std * sigma)
# #     # newmax = band_stats.mean + (band_stats.std * sigma)
# #     newmin = 0
# #     newmax = 2500
# #     style_list.append(
# #         {'band': bid, 'palette': pid, 'min': newmin, 'max': newmax})
# #     print(style_list)

# toa = get_leaflet_tile_layer(
#         client, show=False, band=data_bands,
#         scheme='linear', dtype='uint16', style={'bands': style_list}
# )

# # toa = get_leaflet_tile_layer(client)

# m = Map(
#     center=client.center(), zoom=client.default_zoom, basemap=basemaps.Esri.WorldImagery, scroll_wheel_zoom=True)
# m.add_layer(toa)
# control = LayersControl(position='topright')
# m.add_control(control)
# m.add_control(FullScreenControl())

# m